In [1]:
import json
import subprocess #Need to clean up GPU memory after execution so forced to restart interpreter
import numpy as np

In [2]:
def find_omegas(dat_lines, start_idx, end_idx, entry_line):
    METADATA_LINE_LEN = 51 
    SCAN_ID_IDX = 7
    OMEGA_IDX = 31

    omegas = np.zeros(end_idx - start_idx + 1)
    found_omegas = np.zeros(end_idx - start_idx + 1)

    for i in range(entry_line, 0, -1):
        line = dat_lines[i].split(' ')
        if len(line) == METADATA_LINE_LEN:
            scan_num = int(line[SCAN_ID_IDX])

            if scan_num < start_idx:
                break

            if scan_num <= end_idx:
                store_idx = scan_num - start_idx
                omegas[store_idx] = float(line[OMEGA_IDX])
                found_omegas[store_idx] = True

    if not np.all(found_omegas):
        raise ValueError(f"Unable to find all omegas for given scan range {start_idx}-{end_idx}")    
    
    return omegas

In [3]:
scans = []
NUM_FIELD_BEGIN = 10
NUM_FIELD_END = 20


with open('working_dir/faber_sam1_s12_crack_tomo_TomoFastScan.dat') as dat_f:
    read_counter = 0
    cur_scan = {}
    dat_lines = dat_f.readlines()

    for i, line in enumerate(dat_lines):
        if line.startswith('End'):
            read_counter = 2
        elif read_counter == 2:
            read_counter -= 1
            cur_scan['start'] = int(line.split(' ')[4].strip()) + NUM_FIELD_BEGIN
        elif read_counter == 1:
            read_counter -= 1
            cur_scan['end'] = int(line.split(' ')[4].strip()) - NUM_FIELD_END
            cur_scan['omega'] = find_omegas(dat_lines, 
                                             cur_scan['start'],
                                             cur_scan['end'],
                                             i)
            scans.append(cur_scan)
            cur_scan = {}
            

In [4]:
with open('working_dir/config.json', 'r') as config_f:
    config = json.load(config_f)

In [5]:
for scan in scans:
    print(f"{scan['start'], scan['end']}")

(133706, 135506)
(135537, 137337)
(137368, 139168)
(139200, 141000)
(141031, 142831)
(142862, 144662)
(144694, 146494)
(146525, 148325)
(148356, 150156)
(150188, 151988)
(152019, 153819)
(153850, 155650)
(155682, 157482)
(157513, 159313)
(159344, 161144)
(161177, 162977)
(163008, 164808)
(164839, 166639)
(166670, 168470)
(168501, 170301)
(170332, 172132)
(172163, 173963)
(173994, 175794)


In [6]:
OUT_DIR = 'working_dir/voids_remap' 

error_scans = []
for i, scan in enumerate(scans):
    assert scan['end'] - scan['start'] == len(config['omega'])
    config['img_range'] = [scan['start'], scan['end']]
    config['omega'] = scan['omega']
    curr_config_fp = 'working_dir/temp_all_config.json'
    with open(curr_config_fp, 'w') as curr_conf_f:
        json.dump(config, curr_conf_f)
    map_proc = subprocess.run(['python', 'coarse_mesh.py', f'{curr_config_fp}', OUT_DIR])
    if map_proc.returncode != 0:
        error_scans.append(f"{scan['start']}-{scan['end']}")

    print(f'Finished scan {i + 1}/{len(scans)}')

Traceback (most recent call last):
  File "/home/phoebus3/MPRINCE/Desktop/projects/1IDTomo2Mesh/coarse_mesh.py", line 8, in <module>
    from tomo2mesh.projects.steel_am.coarse2fine import coarse_map, process_subset
  File "/home/phoebus/MPRINCE/anaconda3/envs/tomo2meshuni/lib/python3.9/site-packages/tomo2mesh/__init__.py", line 8, in <module>
    from tomo2mesh.structures.grid import Grid
  File "/home/phoebus/MPRINCE/anaconda3/envs/tomo2meshuni/lib/python3.9/site-packages/tomo2mesh/structures/grid.py", line 16, in <module>
    import cupy as cp
  File "/home/phoebus/MPRINCE/anaconda3/envs/tomo2meshuni/lib/python3.9/site-packages/cupy/__init__.py", line 556, in <module>
    from cupy._math.trigonometric import arccos  # NOQA
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", l

KeyboardInterrupt: 